In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(200_000_000_000)

import intake
import geopandas as gpd
import pandas as pd
from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
from calitp_data_analysis import geography_utils

from shared_utils import webmap_utils, gtfs_utils_v2, rt_dates
import branca

In [47]:
from update_vars import analysis_date, GCS_FILE_PATH, INTERSECTION_BUFFER_METERS

In [3]:
gcsgp = GCSGeoPandas()

In [4]:
analysis_date

'2025-11-05'

In [5]:
catalog = intake.open_catalog('catalog.yml')

## Rail/Ferry/BRT Major Stops

In [6]:
ca = catalog.ca_boundary.read().to_crs(geography_utils.CA_NAD83Albers_m)

In [7]:
rail_ferry_brt = catalog.rail_brt_ferry_stops.read()

In [8]:
rail_ferry_brt = rail_ferry_brt.clip(ca)

In [9]:
hqta_types = rail_ferry_brt.hqta_type.unique()

In [10]:
cmap = branca.colormap.step.Accent_08

In [11]:
color_dict = webmap_utils.categorical_cmap(cmap, hqta_types)

In [12]:
color_dict

{'major_stop_brt': (127, 201, 127),
 'major_stop_ferry': (190, 174, 212),
 'major_stop_rail': (253, 192, 134)}

In [13]:
rail_ferry_brt = rail_ferry_brt.assign(color = rail_ferry_brt.hqta_type.map(color_dict))

In [14]:
webmap_utils.set_state_export(rail_ferry_brt, filename = 'rail_ferry_brt3', cache_seconds=0)

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:128: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


file already exists at calitp-map-tiles/testing/rail_ferry_brt3.geojson.gz, not overwriting, use overwrite = True to overwrite


{'state_dict': {'name': 'null',
  'layers': [{'name': 'Map',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/rail_ferry_brt3.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}}],
  'lat_lon': (36.787838477122335, -121.16992878911985),
  'zoom': 13},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJNYXAiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL3Rlc3RpbmcvcmFpbF9mZXJyeV9icnQzLmdlb2pzb24uZ3oiLCAicHJvcGVydGllcyI6IHsic3Ryb2tlZCI6IGZhbHNlLCAiaGlnaGxpZ2h0X3NhdHVyYXRpb25fbXVsdGlwbGllciI6IDAuNX19XSwgImxhdF9sb24iOiBbMzYuNzg3ODM4NDc3MTIyMzM1LCAtMTIxLjE2OTkyODc4OTExOTg1XSwgInpvb20iOiAxM30='}

In [15]:
rail_ferry_brt

,schedule_gtfs_dataset_key_primary,stop_id,stop_name,route_id,route_type,hqta_type,geometry,color
778,1165b1474df778cb0fc3ba9246e32035,SNS,Salinas,36924,2,major_stop_rail,POINT (-147851.418 -147353.146),"(253, 192, 134)"
1055,118988ea737b00fd3b3129187c8751bc,70322,Gilroy Caltrain Station Southbound,South County,2,major_stop_rail,POINT (-139178.464 -111445.406),"(253, 192, 134)"
1054,118988ea737b00fd3b3129187c8751bc,70321,Gilroy Caltrain Station Northbound,South County,2,major_stop_rail,POINT (-139166.194 -111439.717),"(253, 192, 134)"
1052,118988ea737b00fd3b3129187c8751bc,70311,San Martin Caltrain Station Northbound,South County,2,major_stop_rail,POINT (-142918.111 -102295.656),"(253, 192, 134)"
1053,118988ea737b00fd3b3129187c8751bc,70312,San Martin Caltrain Station Southbound,South County,2,major_stop_rail,POINT (-142994.142 -102135.611),"(253, 192, 134)"
...,...,...,...,...,...,...,...,...
1138,1451f537bdcefd0e8ba827d12c4ef4b8,13858,California St & Divisadero St,1,3,major_stop_brt,POINT (-214642.193 -22613.835),"(127, 201, 127)"
1141,1451f537bdcefd0e8ba827d12c4ef4b8,13885,California St & Pierce St,1,3,major_stop_brt,POINT (-214284.021 -22582.128),"(127, 201, 127)"
1140,1451f537bdcefd0e8ba827d12c4ef4b8,13884,California St & Pierce St,1,3,major_stop_brt,POINT (-214343.273 -22573.598),"(127, 201, 127)"
2764,ff345d53ce8a0e1faf13973402a1dc22,43007,Tiburon (No Service to Angel Island),TRIA,4,major_stop_ferry,POINT (-215721.361 -13173.100),"(190, 174, 212)"


## Analysis Segments and Key Stops

* note that we drop circuitous segments for ease of visualization

In [16]:
hqta_segments = catalog.hqta_segments.read()

In [17]:
path = f'{GCS_FILE_PATH}all_bus.parquet'

In [18]:
path

'gs://calitp-analytics-data/data-analyses/high_quality_transit_areas/all_bus.parquet'

In [19]:
max_arrivals_by_stop = pd.read_parquet(f"{GCS_FILE_PATH}max_arrivals_by_stop.parquet")

In [21]:
gdf = gcsgp.read_parquet(path)

stops = gcsgp.read_parquet(f"{GCS_FILE_PATH}stops_with_lookback.parquet")

stops = stops[['stop_id', 'stop_name', 'analysis_date',
      'schedule_gtfs_dataset_key', 'analysis_name', 'geometry']]

stops = stops.rename(columns={'geometry': 'stop_geometry'})

gdf = gdf.merge(stops, on = ['stop_id', 'schedule_gtfs_dataset_key'])
gdf = gdf[~gdf['circuitous_segment']]

In [42]:
map1 = gdf.copy()[['route_id', 'stop_id', 'geometry',
   'fwd_azimuth_360', 'circuitous_segment', 'hq_transit_corr',
   'ms_precursor', 'analysis_name']]

In [43]:
# Source - https://stackoverflow.com/a
# Posted by mkrieger1, modified by community. See post 'Timeline' for change history
# Retrieved 2025-12-08, License - CC BY-SA 4.0

azimuth_cmap = branca.colormap.LinearColormap(
        colors=list(branca.colormap.linear.viridis.colors) + list(reversed(branca.colormap.linear.viridis.colors)),
        vmin=0, vmax=360
)  # this will correctly show 0 and 360 as close together
azimuth_cmap.caption = '360-degree azimuth (heading)'
azimuth_cmap

In [44]:
webmap_utils.export_legend(azimuth_cmap, 'azimuth_viridis_360a.svg', inner_labels=list(range(72, 360, 72)))

legend written to calitp-map-tiles/azimuth_viridis_360a.svg, public_url https://storage.googleapis.com/calitp-map-tiles/azimuth_viridis_360a.svg


In [45]:
segment_state = webmap_utils.set_state_export(map1, filename = 'hqta_segments', cache_seconds=0,
                                     map_title='HQTA Segments', overwrite=True, color_col='fwd_azimuth_360',
                                     cmap = azimuth_cmap, legend_url='https://storage.googleapis.com/calitp-map-tiles/azimuth_viridis_360a.svg')

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:128: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


In [26]:
map2 = gdf.copy()[['stop_id', 'stop_name', 'am_max_trips_hr',
                  'pm_max_trips_hr', 'route_dir_count', 'analysis_name',
                  'stop_geometry']].set_geometry('stop_geometry')

In [27]:
# map2['color'] = [cmap_dict['stop']] * len(map2)
map2['color'] = [(0,0,0)] * len(map2)

In [67]:
segment_stop_state = webmap_utils.set_state_export(map2, filename = 'hqta_segment_key_stops', cache_seconds=0,
                             existing_state=segment_state, map_title='Key Stops and Segments', overwrite=True,
                                                  manual_centroid=[37.336813156889704, -121.88911054161129])

In [68]:
segment_stop_state

{'state_dict': {'name': 'null',
  'layers': [{'name': 'HQTA Segments',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/hqta_segments.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}},
   {'name': 'Key Stops and Segments',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/hqta_segment_key_stops.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}}],
  'lat_lon': [37.336813156889704, -121.88911054161129],
  'zoom': 13,
  'legend_url': 'https://storage.googleapis.com/calitp-map-tiles/azimuth_viridis_360a.svg'},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJIUVRBIFNlZ21lbnRzIiwgInVybCI6ICJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY2FsaXRwLW1hcC10aWxlcy90ZXN0aW5nL2hxdGFfc2VnbWVudHMuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41fX0sIHsibmFtZSI6ICJLZXkgU3

## Intersecting Segments

In [30]:
pairs = pd.read_parquet(f"{GCS_FILE_PATH}pairwise.parquet")

## Spatial Intersections

In [33]:
intersect = gcsgp.read_parquet(f"{GCS_FILE_PATH}all_intersections.parquet")

In [34]:
by_segment = intersect.dissolve(['hqta_segment_id']).reset_index(drop=False)

In [36]:
by_segment['color'] = [(0, 0, 0)] * len(by_segment)

In [40]:
segment_intersect_state = webmap_utils.set_state_export(by_segment, filename = 'hqta_intersection_areas', cache_seconds=0,
                             existing_state=segment_state, map_title='Segments with Intersections', overwrite=True,
                                                       manual_centroid=[37.336813156889704, -121.88911054161129])

In [41]:
segment_intersect_state

{'state_dict': {'name': 'null',
  'layers': [{'name': 'HQTA Segments',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/hqta_segments.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}},
   {'name': 'Segments with Intersections',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/hqta_intersection_areas.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}}],
  'lat_lon': [37.336813156889704, -121.88911054161129],
  'zoom': 13,
  'legend_url': 'https://storage.googleapis.com/calitp-map-tiles/azimuth_viridis_360a.svg'},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJIUVRBIFNlZ21lbnRzIiwgInVybCI6ICJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY2FsaXRwLW1hcC10aWxlcy90ZXN0aW5nL2hxdGFfc2VnbWVudHMuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41fX0sIHsibmFtZSI6ICJT

## Intersection Buffers and Stop Groups

In [51]:
by_segment.geometry = by_segment.buffer(INTERSECTION_BUFFER_METERS)

In [52]:
major_bus_spatial = gcsgp.read_parquet(f"{GCS_FILE_PATH}major_stop_bus.parquet")

In [63]:
intersect_buffered_state = webmap_utils.set_state_export(by_segment, filename = 'intersect_buffered', cache_seconds=0,
                           map_title='Intersecton Buffers', overwrite=True,
                                                         manual_centroid=[37.336813156889704, -121.88911054161129])

In [64]:
major_bus_spatial['color'] = [(200, 200, 255)] * len(major_bus_spatial)

In [65]:
intersect_major_state = webmap_utils.set_state_export(major_bus_spatial, filename = 'major_bus_spatial', cache_seconds=0,
                           existing_state=intersect_buffered_state, map_title='Buffered Intersections and Stop Groups', overwrite=True,
                                                         manual_centroid=[37.336813156889704, -121.88911054161129])

In [66]:
intersect_major_state

{'state_dict': {'name': 'null',
  'layers': [{'name': 'Intersecton Buffers',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/intersect_buffered.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}},
   {'name': 'Buffered Intersections and Stop Groups',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/major_bus_spatial.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}}],
  'lat_lon': [37.336813156889704, -121.88911054161129],
  'zoom': 13},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJJbnRlcnNlY3RvbiBCdWZmZXJzIiwgInVybCI6ICJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY2FsaXRwLW1hcC10aWxlcy90ZXN0aW5nL2ludGVyc2VjdF9idWZmZXJlZC5nZW9qc29uLmd6IiwgInByb3BlcnRpZXMiOiB7InN0cm9rZWQiOiBmYWxzZSwgImhpZ2hsaWdodF9zYXR1cmF0aW9uX211bHRpcGxpZXIiOiAwLjV9fSwgeyJuYW1lIjogIkJ1ZmZlcmVkIEludGVyc2VjdGlvbnMgYW5kIFN0b3AgR3JvdXBzIiwgInVybCI6